In [ ]:
!nvidia-smi

In [ ]:
!pip install ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.7/698.7 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.1.0-py3-none-any.whl size=20583 sha256=8efec16addf0b39fdf16a1611d535d9809cbe90775d7e23bd7c05065cfd8b6f2
  Stored in directory: /root/.cache/pip/wheels/d8/2d/7a/97a039fca211fa789bffad50ff97dca13c01e9b83e8879f503
Successfully built pyngrok


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------+

# 0. Import des modules et librairies

In [ ]:
import pickle
from os import listdir
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib.image import imread

from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.callbacks import EarlyStopping

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Flatten
from keras import Model

import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

# 1. Chargement des données d'images de masque et sans masque

In [ ]:
base_path = '/content/drive/MyDrive/Mask_Data-20230906T061058Z-001(2)/Mask_Data'
with_mask_path = os.path.join(base_path, 'with_mask')
without_mask_path = os.path.join(base_path, 'without_mask')

# Creation d'une liste contenant les chemins complets de toutes les images présentes dans les deux répertoir
with_mask_images = [os.path.join(with_mask_path, img) for img in os.listdir(with_mask_path)]
without_mask_images = [os.path.join(without_mask_path, img) for img in os.listdir(without_mask_path)]

FileNotFoundError: ignored

# 2. Création d'un modèle de classification d'images basé sur VGG16 avec des couches personnalisées

In [ ]:
# Charger le modèle VGG16 sans les couches fully connected
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Ajouter vos propres couches fully connected
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 classes : avec et sans masque

58889256/58889256 [==============================] - 3s 0us/step


#  3. Vérification des Imports et Affichage d'Exemples d'Images

In [ ]:
# verifier si tout a etait bien import correctement

# Vérifier le nombre d'images importées
print(f"Nombre d'images avec masque : {len(with_mask_images)}")
print(f"Nombre d'images sans masque : {len(without_mask_images)}")

print("Quelques exemples d'images avec masque :")
for img_path in with_mask_images[:5]:  # Afficher les 5 premiers
    print(os.path.basename(img_path))

print("\nQuelques exemples d'images sans masque :")
for img_path in without_mask_images[:5]:  # Afficher les 5 premiers
    print(os.path.basename(img_path))


Nombre d'images avec masque : 755
Nombre d'images sans masque : 753
Quelques exemples d'images avec masque :
augmented_image_151.jpg
267-with-mask.jpg
augmented_image_113.jpg
augmented_image_235.jpg
15-with-mask.jpg

Quelques exemples d'images sans masque :
440.jpg
399.jpg
447.jpg
284.jpg
477.jpg


# 4. Préparation des Données et Entraînement du Modèle de Classification

In [ ]:
# Création des étiquettes (0 pour avec masque, 1 pour sans masque)
with_mask_labels = np.zeros(len(with_mask_images))
without_mask_labels = np.ones(len(without_mask_images))

# Fusion des images et des étiquettes
all_images = with_mask_images + without_mask_images
all_labels = np.concatenate((with_mask_labels, without_mask_labels))

# Conversion des étiquettes en catégories (one-hot encoding)
all_labels = to_categorical(all_labels, num_classes=2)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.2, random_state=42)

# Fonction de prétraitement pour les images
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

# Prétraiter les images d'entraînement et de test
X_train = np.vstack([preprocess_image(img) for img in X_train])
X_test = np.vstack([preprocess_image(img) for img in X_test])

# Entraîner le modèle
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Utilisation d'un EarlyStopping pour éviter le surapprentissage
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

# Entraîner le modèle
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, callbacks=[early_stopping])


NameError: ignored

In [ ]:
model.save('mask_detection_model.h5')

# 5. Camera

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# Votre fonction pour détecter le masque
def detecter_masque_depuis_camera():
    # Charger le modèle pré-entraîné
    model = load_model('mask_detection_model.h5')

    # Commencer à capturer la vidéo depuis la caméra par défaut
    cap = cv2.VideoCapture(0)
    cv2.namedWindow("Test")
    img_counter = 0

    if not cap.isOpened():
        print("La caméra n'a pas pu être ouverte.")
    else:
        while True:
            # Capturer l'image image par image
            ret, frame = cap.read()

            # Afficher la trame résultante
            cv2.imshow('Caméra en Direct', frame)
            cv2.waitKey(1)

            # Prétraiter la trame pour la prédiction
            img = cv2.resize(frame, (224, 224))
            img = img_to_array(img)
            img = np.expand_dims(img, axis=0)
            img = preprocess_input(img)

            # Prédire si une personne porte un masque ou non
            prediction = model.predict(img)

            # Afficher le résultat sur la trame
            if prediction[0][0] > prediction[0][1]:
                texte = "Masque : Oui"
            else:
                texte = "Masque : Non"

            cv2.putText(frame, texte, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Quitter la boucle si 'q' est pressé
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Lorsque tout est terminé, libérer la capture
        cap.release()
        cv2.destroyAllWindows()

# Votre fonction pour prendre une photo
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename


In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
# Fonction pour capturer la vidéo depuis la caméra et l'analyser pour détecter les masques
def detecter_masque_depuis_camera():
    # Charger le modèle pré-entraîné
    model = load_model('mask_detection_model.h5')

    # Commencer à capturer la vidéo depuis la caméra par défaut
    cap = cv2.VideoCapture(0)
    cv2.namedWindow("Test")
    img_counter = 0


    if not capture.isOpened():
      print("La caméra n'a pas pu être ouverte.")
    else:
      while True:
        # Capturer l'image image par image
        ret, frame = cap.read()

        # Afficher la trame résultante
        cv2.imshow('Caméra en Direct', frame)
        cv2.waitKey(1)

        # Prétraiter la trame pour la prédiction
        img = cv2.resize(frame, (224, 224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)

        # Prédire si une personne porte un masque ou non
        prediction = model.predict(img)

        # Afficher le résultat sur la trame
        if prediction[0][0] > prediction[0][1]:
            texte = "Masque : Oui"
        else:
            texte = "Masque : Non"

        cv2.putText(frame, texte, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Quitter la boucle si 'q' est pressé
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Lorsque tout est terminé, libérer la capture
    cap.release()
    cv2.destroyAllWindows()

# 6. Flask


In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://vb1dkc08ht8-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [ ]:
from flask import Flask, request, render_template


app = Flask(__name__)

@app.route('/detection_Mask')
def detecter_masque():
    # Charger le modèle pré-entraîné
    model = load_model('mask_detection_model.h5')

    # Capture vidéo depuis la caméra
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()

        # Prétraitement de l'image
        img = cv2.resize(frame, (224, 224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)

        # Prédiction
        prediction = model.predict(img)

        # Afficher le résultat sur la trame
        if prediction[0][0] > prediction[0][1]:
            texte = "Masque : Oui"
        else:
            texte = "Masque : Non"

        cv2.putText(frame, texte, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Afficher la trame résultante
        cv2.imshow('Trame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    return "Processus de détection terminé"

if __name__== "__main__":
  app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
